# Translating PydanticAI Agent to OpenAI Agents SDK

We will illustrate the concept of guardrails with OpenAI Agents SDK because it has built-in support. With Pydantic AI we need to implement guardrails ourselves. We will do that later.

First we will translate the agent we created so far into Agents SDK.

This will prepare us for implementing guardrails in the next step.

If you want to skip this video, you can find the complete code here: https://github.com/alexeygrigorev/ai-bootcamp-codespace/tree/main/week4/guardrails-agents-sdk.

## Setting Up the Agent

We will reuse the documentation and search tools from our previous PydanticAI implementation.

Copy the docs and `search_tools` modules from the previous agent.

In [ ]:
import docs
import search_tools

We also need to import the dataclasses and Pydantic models.

In [ ]:
from dataclasses import dataclass
from pydantic import BaseModel

Now copy things from the `search_agent` module:

First, the agent configuration with chunk sizes and model parameters.

In [ ]:
@dataclass
class AgentConfig:
    chunk_size: int = 2000
    chunk_step: int = 1000
    top_k: int = 5

    model: str = "gpt-4o-mini"


Next, the instructions:

In [ ]:
search_instructions = """
You are a search assistant for the Evidently documentation.

Evidently is an open-source Python library and cloud platform for evaluating, testing, and monitoring data and AI systems.
It provides evaluation metrics, testing APIs, and visual reports for model and data quality.

Your task is to help users find accurate, relevant information about Evidently's features, usage, and integrations.

You have access to the following tools:

- search — Use this to explore the topic and retrieve relevant snippets or documentation.
- read_file — Use this to retrieve or verify the complete content of a file when:
    * A code snippet is incomplete, truncated, or missing definitions.
    * You need to check that all variables, imports, and functions referenced in code are defined.
    * You must ensure the code example is syntactically correct and runnable.

If `read_file` cannot be used or the file content is unavailable, clearly state:
> "Unable to verify with read_file."

Search Strategy

- For every user query:
    * Perform at least 3 and at most 6 distinct searches to gather enough context.
    * Each search must use a different phrasing or keyword variation of the user's question.
    * Make sure that the search requests are relevant to evidently, testing, evaluating and monitoring AI systems.
    * No need to include "Evidently" in the search text.

- After collecting search results:
    1. Synthesize the information into a concise, accurate answer.
    2. If your answer includes code, always validate it with `read_file` before finalizing.
    3. If a code snippet or reference is incomplete, explicitly mention it.

Important:
- The 6-search limit applies only to `search` calls.
- You may call `read_file` at any time, even after the search limit is reached.
- `read_file` calls are verification steps and do not count toward the 6-search limit.

Code Verification and Completeness Rules

- All variables, functions, and imports in your final code examples must be defined or imported.
- Never shorten, simplify, or truncate code examples. Always present the full, verified version.
- When something is missing or undefined in the search results:
    * Call `read_file` with the likely filename to retrieve the complete file content.
    * Replace any partial code with the full verified version.
- If the file is not available or cannot be verified:
    * Include a clear note: "Unable to verify this code."
- Do not reformat, rename variables, or omit lines from the verified code.

Output Format

- Write your answer clearly and accurately.
- Include a "References" section listing the search queries or file names you used.
- If you couldn't find a complete answer after 6 searches, set found_answer = False.
"""


Finally, the output models:

In [ ]:
class Reference(BaseModel):
    title: str
    filename: str

class Section(BaseModel):
    heading: str
    content: str
    references: list[Reference]

class SearchResultArticle(BaseModel):
    found_answer: bool
    title: str
    sections: list[Section]
    references: list[Reference]


Now we can implement the agent.

## Creating the Agent

Initialize the configuration and prepare the search tools.

In [ ]:
config = AgentConfig()

tools = search_tools.prepare_search_tools(
    config.chunk_size,
    config.chunk_step,
    config.top_k
)


Create the agent with the tools and instructions.

In [ ]:
from agents import Agent, function_tool

agent_tools = [
    function_tool(tools.search),
    function_tool(tools.read_file)
]

search_agent = Agent(
    name='search',
    tools=agent_tools,
    instructions=search_instructions,
    model=config.model,
    output_type=SearchResultArticle,
)


## Running the Agent

Run the agent with a simple query.

In [ ]:
from agents import Runner

input = 'llm as a judge'
result = await Runner.run(search_agent, input=input)

## Streaming the Response

We can stream the agent response to see tool calls and output in real time.

In [ ]:
from openai.types.responses import ResponseTextDeltaEvent

result = Runner.run_streamed(
    search_agent,
    input=input,
)

async for event in result.stream_events():
    if event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            tool_call = event.item.raw_item
            f_name = tool_call.name
            args = tool_call.arguments
            print(f"TOOL CALL ({event.item.agent.name}): {f_name}({args})")
    
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)


## Parsing Structured Output

Like previously, we can parse the JSON output incrementally as it streams.

In [ ]:
from jaxn import StreamingJSONParser, JSONParserHandler

class SearchResultHandler(JSONParserHandler):
    def on_field_start(self, path: str, field_name: str):
        if field_name == "references":
            level = path.count("/") + 2
            print(f"\n{'#' * level} References\n")

    def on_field_end(self, path, field_name, value, parsed_value=None):
        if field_name == "title" and path == "":
            print(f"# {value}")

        elif field_name == "heading":
            print(f"\n\n## {value}\n")
        elif field_name == "content":
            print("\n") 

    def on_value_chunk(self, path, field_name, chunk):
        if field_name == "content":
            print(chunk, end="", flush=True)

    def on_array_item_end(self, path, field_name, item=None):
        if field_name == "references":
            title = item.get("title", "")
            filename = item.get("filename", "")
            print(f"- [{title}]({filename})")

handler = SearchResultHandler()
parser = StreamingJSONParser(handler)

# Parse each chunk as it arrives
parser.parse_incremental(event.data.delta)


## Handling Too Many Tool Calls

The agent might exceed the maximum number of tool calls. In PydanticAI we used `history_processors` for that.

In Agents SDK we have a little less flexibility, but we can count the number of iterations of the loop, and break the execution when it exceeds the limit.

We do it by setting the `max_turns` parameter. This is how we can do it:



In [ ]:
from agents.exceptions import MaxTurnsExceeded

try:
    # Run the agent
    result = Runner.run_streamed(
        search_agent,
        input=input,
        max_turns=3
    )

    ...
except MaxTurnsExceeded as e:
    print('too many turns')
    finish_prompt = 'System message: The number of searches has exceeded the limit. Proceed to finishing the writeup'
    finish_message = [{'role': 'user', 'content': finish_prompt}]
    messages = result.to_input_list() + finish_message
    # Run one more time with the accumulated messages


## Complete Streaming Function

Here is the complete function that handles streaming and turn limits.

In [ ]:
async def run_stream(agent, input, handler, max_turns=3):
    try:
        result = Runner.run_streamed(
            agent,
            input=input,
            max_turns=max_turns
        )
        
        parser = StreamingJSONParser(handler)

        async for event in result.stream_events():
            if event.type == "run_item_stream_event":
                if event.item.type == "tool_call_item":
                    tool_call = event.item.raw_item
                    f_name = tool_call.name
                    args = tool_call.arguments
                    print(f"TOOL CALL ({event.item.agent.name}): {f_name}({args})")
            
            if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
                parser.parse_incremental(event.data.delta)

        return result
    except MaxTurnsExceeded as e:
        print('too many turns')
        finish_prompt = 'System message: The number of searches has exceeded the limit. Proceed to finishing the writeup'
        finish_message = [{'role': 'user', 'content': finish_prompt}]
        messages = result.to_input_list() + finish_message
        final_result = await run_stream(agent, input=messages, handler=handler, max_turns=1)
        return final_result


Let's run it:

In [ ]:
result = await run_stream(search_agent, 'llm as a judge', SearchResultHandler())

Now we're ready to add guardrails to this code.